# 環境設定
  - 標準コード1と同様、まずは環境設定を行います。

In [ ]:
# ============================================================
# 0) 依存関係の固定（Colabの“環境ブレ”対策）
# ============================================================
# Colab（無料版）は、ある日突然プリインストール版が変わり、
# それまで動いていた学習コードが壊れることが頻繁にあります。
# そのため、このセルでは「一度全部消す → 互換が確認できたバージョンを入れ直す」
# という“強制的な再現性確保”をしています。
#
# やりがちな事故：
# - 既に入っているパッケージと混ざって「importは通るが実行で落ちる」
# - transformers / trl / unsloth の相性不一致で、謎エラーや速度低下が起きる
#
# ※このセルは“おまじない”ではなく「再現性のための重要工程」です。

#!pip -q uninstall -y numpy pandas datasets trl transformers accelerate peft unsloth unsloth-zoo bitsandbytes xformers
!uv pip install "numpy==2.0.2" "pandas==2.2.2"

# unsloth-zoo が要求する範囲に合わせる
# ここでは Unsloth と相性の良い transformers / trl / accelerate / peft / bitsandbytes を固定します。
# 特に transformers は細かいバージョン差で挙動が変わりやすいので固定が重要です。
!uv pip install \
  "datasets==4.3.0" \
  "trl==0.24.0" \
  "transformers==4.56.2" \
  "accelerate==1.4.0" \
  "peft==0.13.2" \
  "bitsandbytes==0.45.0"

# unsloth / zoo を同系列で揃える（zoo側の要求に合わせる）
# Unsloth本体と unsloth-zoo は“セット運用”が基本です。片方だけ上げると壊れがちです。
!uv pip install "unsloth-zoo==2025.12.7" "unsloth==2025.12.7"



Using Python 3.12.12 environment at: /usr
Audited 2 packages in 231ms
Using Python 3.12.12 environment at: /usr
Resolved 68 packages in 463ms
Prepared 7 packages in 1.70s
Uninstalled 5 packages in 393ms
Installed 7 packages in 71ms
 - accelerate==1.12.0
 + accelerate==1.4.0
 + bitsandbytes==0.45.0
 - datasets==4.0.0
 + datasets==4.3.0
 - peft==0.18.1
 + peft==0.13.2
 - pyarrow==18.1.0
 + pyarrow==23.0.0
 - transformers==4.57.6
 + transformers==4.56.2
 + trl==0.24.0
Using Python 3.12.12 environment at: /usr
Resolved 86 packages in 329ms
Prepared 8 packages in 2.01s
Uninstalled 2 packages in 52ms
Installed 8 packages in 13ms
 - bitsandbytes==0.45.0
 + bitsandbytes==0.49.1
 + cut-cross-entropy==25.1.1
 + msgspec==0.20.0
 - torchao==0.10.0
 + torchao==0.15.0
 + tyro==1.0.5
 + unsloth==2025.12.7
 + unsloth-zoo==2025.12.7
 + xformers==0.0.33.post1


# モデルとトークナイザーのロード（4bit量子化）
#### 何をしているか
- Qwen/Qwen3-4B-Instruct-2507 という事前学習済みモデルを読み込みます。
- tokenizer は「文章をトークン（ID列）に変換する道具」です。モデル学習・推論どちらでも必須です。
- DPO 学習を「モデル全部の重み」を更新してやると計算量も保存容量も大きくなります。
- そこで LoRA（Low-Rank Adaptation） を使い、モデル内部の一部の層（ここでは attention / MLP 系の投影層）に対して「追加の小さな学習可能パラメータ」を挿入し、そこだけ更新します。
#### 重要ポイント

- load_in_4bit = True により 4bit量子化でモデルを読み込み、GPU メモリを節約します。
- 4B（40億パラメータ級）のモデルでも、Colab の T4 など比較的メモリが少ない環境で動かしやすくなります。
- max_seq_length = 2048 は、モデルが扱う最大長（概念的には「一度に読める長さ」）の上限を指定します。

In [ ]:

# -----------------------------
# HF login (once)
# -----------------------------
#「HF Hub上のデータセットを読む」「学習したモデルをHFにアップする」ためにログインします。
#

from unsloth import FastLanguageModel
import numpy as np, pandas as pd
import datasets, trl, transformers, torch

from huggingface_hub import login, HfApi
login()  # Colab will prompt
api = HfApi()

In [ ]:
import sys
import os
import warnings
from unsloth import FastLanguageModel
import torch

# 1. Load Model & Tokenizer
max_seq_length = 2048
dtype = None # Auto detection
load_in_4bit = True # Use 4bit quantization

print("Loading model (Qwen/Qwen3-4B-Instruct-2507)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-4B-Instruct-2507",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. Configure LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("Model loaded and LoRA adapters applied successfully.")

# DPO用データセットのロードと整形

#### 何をしているか
1. チャットテンプレートの設定（モデル入力の“型”を統一）
    - チャット系モデルは、内部的には「ユーザ発話」「アシスタント発話」などの区切り（特殊トークン）を含む決まった形式で学習されています。
    - chat_template="qwen-2.5" を指定することで、入力を Qwen 互換のチャット形式に整形するルールを tokenizer に持たせます。
2. DPO用データセットのロード
    - Hugging Face 上の u-10bei/dpo-dataset-qwen-cot データセットを読み込み、train split を取得します。
    - 提供DPOデータセット
      https://huggingface.co/datasets/u-10bei/dpo-dataset-qwen-cot

    DPO 用データは基本的に次の3列を持ちます：
    - prompt：質問や指示（ユーザ側）
    - chosen：望ましい回答（良い例）
    - rejected：望ましくない回答（悪い例）
3. prompt / chosen / rejected を “チャット形式の文字列”に整形
  
    dataset.map(..., batched=True) でまとめて処理できるように、バッチ（複数行）を受け取り、整形済みの3列を返す関数です。
    
    tokenizer.apply_chat_template(...) が重要で、これにより
    - prompt は「ユーザ発話」として整形され、さらに add_generation_prompt=True により「ここからアシスタントが生成する」という合図（特殊トークン）が末尾に付きます。
    - chosen/rejected は「アシスタント発話」として整形されます。

    ※ なぜ prompt だけ add_generation_prompt=True なのか

        DPO は「同じ prompt に対して chosen のほうが rejected より望ましい」ように学習するので、prompt 側は「アシスタントが続けて答える前提」の形に揃えると扱いやすいからです。
4. 整形関数をデータセットに適用
    - map によって、全データに整形処理を適用します。
    - dataset[0] を出力し、整形後の prompt/chosen/rejected が想定通りのチャット形式になっているか目視確認します。


In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# 1. Setup Chat Template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

# 2. Load Dataset
dataset = load_dataset("u-10bei/dpo-dataset-qwen-cot", split = "train")

# 3. Define Formatting Function
def formatting_prompts_func(examples):
    new_prompts = []
    new_chosens = []
    new_rejecteds = []

    # Iterate over the batch
    for prompt, chosen, rejected in zip(examples['prompt'], examples['chosen'], examples['rejected']):
        # Format prompt with user role and add generation prompt (e.g. <|im_start|>assistant)
        formatted_prompt = tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize = False,
            add_generation_prompt = True
        )

        # Format chosen/rejected responses with assistant role
        formatted_chosen = tokenizer.apply_chat_template(
            [{"role": "assistant", "content": chosen}],
            tokenize = False,
        )

        formatted_rejected = tokenizer.apply_chat_template(
            [{"role": "assistant", "content": rejected}],
            tokenize = False,
        )

        new_prompts.append(formatted_prompt)
        new_chosens.append(formatted_chosen)
        new_rejecteds.append(formatted_rejected)

    return {
        "prompt": new_prompts,
        "chosen": new_chosens,
        "rejected": new_rejecteds,
    }

# 4. Apply Formatting
dataset = dataset.map(formatting_prompts_func, batched = True)

# 5. Verify
print("Dataset columns:", dataset.column_names)
print("\n--- Sample Prompt ---")
print(dataset[0]["prompt"])
print("\n--- Sample Chosen ---")
print(dataset[0]["chosen"])
print("\n--- Sample Rejected ---")
print(dataset[0]["rejected"])

README.md:   0%|          | 0.00/387 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.37M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4040 [00:00<?, ? examples/s]

Map:   0%|          | 0/4040 [00:00<?, ? examples/s]

Dataset columns: ['prompt', 'chosen', 'rejected', 'strategy']

--- Sample Prompt ---
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
<|im_start|>system
You are a helpful assistant. Please format your response as follows:
Approach: <step-by-step reasoning>
Output: <final answer><|im_end|>
<|im_start|>user
Produce a TOML document for a api specification.<|im_end|>
<|im_start|>assistant
<|im_end|>
<|im_start|>assistant


--- Sample Chosen ---
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>assistant
Approach:
1. Task: Create api specification in TOML
2. Complexity: complex - 8-10 fields, 3-4 levels
3. Format rules: key=value syntax, [sections], proper types
4. Structure: Organize data logically with appropriate nesting
5. Populate fields with realistic example data based on the schema.

Output:
openapi = "3.0.0"

[info]
title = "Face-to-face dedicated model API"

# DPOConfig（学習ハイパーパラメータ）の設定とDPOTrainer の初期化

#### 何をしているか
- trl ライブラリ（Transformer Reinforcement Learning）に含まれる DPO 実装に対して、学習条件をまとめて渡します。
- DPO 学習の実行主体（Trainer）を作ります。
    - model：LoRA 適用済みの学習対象モデル
    - tokenizer：チャット形式を扱える tokenizer
    - train_dataset：整形済みの DPO データ
    - args：学習設定

#### ハイパーパラメータの初期設定

- learning_rate=1e-7：
  
      学習率は小さめ
- per_device_train_batch_size=2 と gradient_accumulation_steps=4：
    
      実効バッチサイズは概ね 2 × 4 = 8 相当になります
      GPU メモリが少ない環境のため、小さいバッチにしています

- optim="adamw_8bit"：

      8bit Optimizer（メモリの節約）

- fp16 / bf16：

      GPU が bf16 対応なら bf16 を使い、対応していなければ fp16 を使う分岐

- beta=0.1：

      DPO の重要パラメータで、「chosen と rejected の好み差をどれくらい強く学習するか」の温度係数的な役割を持ちます。

- max_prompt_length=512, max_length=1024：

      prompt 部分は最大 512 トークン、全体（prompt+応答）は最大 1024 トークンまで、という切り詰めの基準です。
      長すぎるデータはここで切られます（学習に入らない部分が出ます）。


In [ ]:
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

# 1. Configure DPO Training Arguments
dpo_config = DPOConfig(
    learning_rate = 1e-7,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    num_train_epochs = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    output_dir = "dpo_checkpoints",
    beta = 0.1,
    max_length = 1024,
    max_prompt_length = 512,
    seed = 42,
    report_to = "none",
)

# 2. Initialize DPOTrainer
trainer = DPOTrainer(
    model = model,
    ref_model = None, # Unsloth handles this implicitly for PEFT
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = dpo_config,
)

print("DPOTrainer initialized.")

# DPOトレーニングの実行
#### 何をしているか
    - trainer.train() で実際の DPO 学習が走ります
    - 学習が終わったモデル（LoRA を含む状態）を dpo_lora_model に保存します
    - trainer_stats には損失や学習ステップなどの統計情報が入ります

In [ ]:
# 1. Adjust logging steps to reduce output verbosity
if hasattr(trainer, 'args'):
    trainer.args.logging_steps = 50

print("Starting DPO training...")
# 2. Run DPO Training
trainer_stats = trainer.train()

# 3. Save the model and tokenizer
output_dir = "dpo_lora_model"
trainer.save_model(output_dir)

# 4. Print Training Stats
print(f"\nTraining completed. Model saved to '{output_dir}'.")
print("Training Statistics:", trainer_stats)

Starting DPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,040 | Num Epochs = 1 | Total steps = 505
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 16,515,072 of 4,038,983,168 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
50,0.675000,0.032653,-0.016905,0.577500,0.049558,-854.382629,-264.011444,-2.953451,-3.451738,0,0,0
100,0.401400,0.653065,-0.127522,0.960000,0.780587,-802.003357,-266.913208,-3.102431,-3.485450,No Log,No Log,No Log
150,0.107700,2.189785,-0.394114,0.997500,2.583899,-753.982178,-262.196014,-2.953183,-3.468762,No Log,No Log,No Log
200,0.020800,4.304415,-0.759475,0.997500,5.063889,-762.066406,-283.616394,-3.050305,-3.500772,No Log,No Log,No Log
250,0.010000,5.768434,-0.910353,0.997500,6.678787,-720.026550,-277.243530,-3.039755,-3.548172,No Log,No Log,No Log
300,0.010400,6.655694,-0.970324,0.997500,7.626018,-766.749939,-284.989868,-2.942729,-3.495684,No Log,No Log,No Log
350,0.001600,7.010244,-1.042516,1.000000,8.052760,-729.103455,-272.656525,-2.936298,-3.532874,No Log,No Log,No Log
400,0.008500,7.128240,-1.065809,0.997500,8.194048,-736.437012,-289.967102,-2.938479,-3.496388,No Log,No Log,No Log
450,0.007700,7.206686,-1.034481,1.000000,8.241167,-730.509399,-293.246002,-3.019848,-3.479598,No Log,No Log,No Log
500,0.005200,7.346134,-1.073525,1.000000,8.419660,-759.381775,-278.812195,-2.903349,-3.487840,No Log,No Log,No Log



Training completed. Model saved to 'dpo_lora_model'.
Training Statistics: TrainOutput(global_step=505, training_loss=0.12364573818400945, metrics={'train_runtime': 5700.8135, 'train_samples_per_second': 0.709, 'train_steps_per_second': 0.089, 'total_flos': 0.0, 'train_loss': 0.12364573818400945, 'epoch': 1.0})


# HuggingFaceへのモデルのアップロード（merged_16bit）
#### 何をしているか
  - whoami API で、自分の Hugging Face ユーザ名を取得します。
  - username/dpo-qwen-cot-merged という形式のリポジトリ名を自動生成して、そこにアップロードします。
  - 本コードでは、SFTの標準コードと異なり、アダプタではなく「マージ済みモデル」をアップロードするようにしています。
  - アップロードにあたり、必要事項を記載したREADME.mdを作成して併せてアップします。

#### ポイント
  - README.mdに誤った内容を記載しないよう、事前に内容を確認しましょう。
  - 【課題】README.mdのモデルタイトルを手書きで記入してください。
  - push_to_hub_merged は、Unsloth が提供する「LoRA をベースモデルにマージして保存・アップロードする」便利関数です。
  - save_method="merged_16bit" なので、16bit でマージ済みモデルとして保存されます。
  - 16bit でマージすると、LoRA 単体よりファイルサイズが大きくなります。
  - ただし、推論時に「ベース + LoRA を別々に読み込む」必要がなくなるため、利用しやすい形式でもあります。

  

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import login, HfApi

# 1. Hugging Face ログイン
# Colabの秘密鍵（🔑）に「HF_TOKEN」を設定していることを前提とします
HF_TOKEN = os.environ.get('HF_TOKEN')
if not HF_TOKEN:
    try:
        HF_TOKEN = userdata.get('HF_TOKEN')
    except:
        print("Warning: HF_TOKEN not found in Secrets.")

if HF_TOKEN:
    login(token=HF_TOKEN, add_to_git_credential=True)
else:
    print("Error: Hugging Face Token is missing. Please check your Colab Secrets.")

# 2. ユーザー名の取得とリポジトリ名の設定
api = HfApi()
try:
    username = api.whoami(token=HF_TOKEN)["name"]
    repo_name = "dpo-qwen-cot-merged"  # 任意のリポジトリ名
    repo_id = f"{username}/{repo_name}"
    print(f"Uploading to: {repo_id}")
except Exception as e:
    print(f"Failed to get username: {e}")


In [ ]:

# 3. README.md（モデルカード）の自動生成

import os

# 保存先ディレクトリの確認
readme_dir = "dpo_lora_model"
os.makedirs(readme_dir, exist_ok=True)

def _fmt_lr(x) -> str:
    try:
        return f"{float(x):.0e}"
    except Exception:
        return str(x)

# 学習パラメーターの取得（自動反映用）
base_model_name = "Qwen/Qwen3-4B-Instruct-2507"
dataset_name = "u-10bei/dpo-dataset-qwen-cot"
lr_str = _fmt_lr(dpo_config.learning_rate)
epochs = dpo_config.num_train_epochs
beta = dpo_config.beta
max_len = dpo_config.max_length

# README内のタイトル（ここを自分で記入する）
title_line = "＜【課題】ここは自分で記入して下さい＞" # 例： qwen3-4b-dpo-qwen-cot-merged

readme_md = f"""---
base_model: {base_model_name}
datasets:
- {dataset_name}
language:
- en
license: apache-2.0
library_name: transformers
pipeline_tag: text-generation
tags:
- dpo
- unsloth
- qwen
- alignment
---

# {title_line}

This model is a fine-tuned version of **{base_model_name}** using **Direct Preference Optimization (DPO)** via the **Unsloth** library.

This repository contains the **full-merged 16-bit weights**. No adapter loading is required.

## Training Objective
This model has been optimized using DPO to align its responses with preferred outputs, focusing on improving reasoning (Chain-of-Thought) and structured response quality based on the provided preference dataset.

## Training Configuration
- **Base model**: {base_model_name}
- **Method**: DPO (Direct Preference Optimization)
- **Epochs**: {epochs}
- **Learning rate**: {lr_str}
- **Beta**: {beta}
- **Max sequence length**: {max_len}
- **LoRA Config**: r=8, alpha=16 (merged into base)

## Usage
Since this is a merged model, you can use it directly with `transformers`.

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "your_id/your-repo-name"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Test inference
prompt = "Your question here"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

```

## Sources & License (IMPORTANT)

* **Training Data**: [{dataset_name}]
* **License**: MIT License. (As per dataset terms).
* **Compliance**: Users must follow the original base model's license terms.
"""

# ファイル書き出し

readme_path = os.path.join(readme_dir, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
  f.write(readme_md)

print(f"✅ README.md を作成しました: {readme_path}")

import os
from huggingface_hub import HfApi


# 4-1. モデル本体のアップロード (Merged 16-bit)
model.push_to_hub_merged(
    repo_id,
    tokenizer,
    save_method = "merged_16bit",
    token = HF_TOKEN
)

# 4-2. 作成した README.md を上書きアップロード
api.upload_file(
    path_or_fileobj=os.path.join("dpo_lora_model", "README.md"),
    path_in_repo="README.md",
    repo_id=repo_id,
    repo_type="model",
)

print(f"✅ 全てのアップロードが完了しました！\nURL: https://huggingface.co/{repo_id}")